In [7]:
import os
import matplotlib.pyplot as plt
import argparse
from utils import *
from models.lstm_classifier import LSTMClassifier
from models.gru_classifier import GRUClassifier
from keras.optimizers import RMSprop

In [2]:
train = load_data('data/utterances.train')
val = load_data('data/utterances.valid')

In [3]:
classes = train['dialog_act'].unique()
num_classes = len(classes)
max_len = train['utterance_t'].apply(len).max()

In [4]:
(X_train, y_train), (vocab, word_to_idx, idx_to_word) = process_data(train, max_len)
(X_val, y_val), (_,_,_) = process_data(val, max_len)

In [5]:
embds_path = '/projekte/slu/share/GoogleNews-vectors-negative300.bin' 
w2v = load_bin_vec(embds_path, word_to_idx)
add_unknown_words(w2v, word_to_idx)
W, word_idx = get_W(w2v)
#W = np.array([weight for weight in W])

In [8]:
model = LSTMClassifier(num_classes, max_len, W)
optimizer = RMSprop(lr=0.01, decay=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 106, 300)          7351800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 106, 128)          219648    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 31)                3999      
Total params: 7,575,447
Trainable params: 7,575,447
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
print(max_len)

106


In [18]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GlobalMaxPooling1D, TimeDistributed
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(W.shape[0], W.shape[1], input_length=max_len,
                   weights=[W], mask_zero=False))
model.add(LSTM(128, dropout=0.2, return_sequences=True,
          kernel_initializer='random_uniform', recurrent_initializer='glorot_uniform'))
model.add(TimeDistributed(Dense(128,
                                input_shape=(max_len, 128))))
model.add(GlobalMaxPooling1D())
model.add(Dense(num_classes, activation='softmax'))

optimizer = RMSprop(lr=0.01, decay=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 106, 300)          7351800   
_________________________________________________________________
lstm_8 (LSTM)                (None, 106, 128)          219648    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 106, 128)          16512     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 31)                3999      
Total params: 7,591,959
Trainable params: 7,591,959
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=3,batch_size=256)

ValueError: Error when checking target: expected dense_6 to have shape (31,) but got array with shape (1,)

In [ ]:
plt.plot(history.history['acc'])